# Pivotting the table 

This notebook transforms raw logging data from `../data/describe.csv` into a format suitable for coding by RA. 

The original table has each row representing a single response in a trial, with multiple rows combining to represent a participant's responses. To facilitate agreement analysis, we pivot the table, where each row corresponds to a participant and each column to a response.

The processed data is stored in`../data/pivot.csv`, pending coding. The coded data is available in `../data/coded_data.xlsx`. 

For analyis we only consider valid participants. They are identified using information from `../data/survey.csv`; only students who completed the post-experiment survey are considered.

In [2]:
import pandas as pd

In [3]:
survey_df = pd.read_csv("../data/survey.csv")
describe_df = pd.read_csv("../data/describe.csv")

In [4]:
uids = set(survey_df["uid"])
len(uids)

97

In [5]:
describe_df = describe_df[describe_df["uid"].isin(uids)]

In [6]:
# Sanity check, every study should have at least 30 trials
for uid in uids:
    n = len(describe_df[describe_df["uid"] == uid])
    assert n >= 30, f"{uid}, len = {n}"

In [8]:
result = pd.pivot_table(
    describe_df,
    index=["uid"],
    columns="object_id",  # we want to convert `object_id` values into its own columns
    values=["description", "seen"],  # columns to keep
    aggfunc="max",  # in our case, each student will only have one description/seen for an object, `max` will give us the value
)
result.head()

description  \
object_id                                                                            1    
uid                                                                                       
019c8ff2-e547-420e-b775-d8089d6b9653                                           Mushroom   
07ff7f5a-73ab-4741-b5c3-127c247b79b7  There is an arch on the top of a half of a rec...   
08d92e08-2e2d-4560-84bf-0b2c352fce93       This is a bridge with an alleyway under it.    
0c6be3bd-e5d0-4deb-90c9-93d53b32c3a7     rainbow shape on top of rounded trapezoid base   
0d51a401-ef06-493f-bccd-0d0e91a53c6d                   It kind of looks like a rainbow.   

                                                                                         \
object_id                                                                            2    
uid                                                                                       
019c8ff2-e547-420e-b775-d8089d6b9653                                                bus   
07ff7f5a-73ab-4741-b5c3-127c247b79b7  sphere with a disk with a cylinder with anothe...   
08d92e08-2e2d-4560-84bf-0b2c352fce93  While it is not exactly similar to the last de...   
0c6be3bd-e5d0-4deb-90c9-93d53b32c3a7  circular prisms connected by cylinder with rou...   
0d51a401-ef06-493f-bccd-0d0e91a53c6d                                It looks a dumbell.   

                                                                                         \
object_id                                                                            3    
uid                                                                                       
019c8ff2-e547-420e-b775-d8089d6b9653                                             dradel   
07ff7f5a-73ab-4741-b5c3-127c247b79b7  hexagonal prism with 3 triangular prisms on on...   
08d92e08-2e2d-4560-84bf-0b2c352fce93  This object consists of two different sized he...   
0c6be3bd-e5d0-4deb-90c9-93d53b32c3a7  hexagonal shape with smaller hexagon sticking ...   
0d51a401-ef06-493f-bccd-0d0e91a53c6d  It is a hexagon with another smaller hexagon o...   

                                                                                         \
object_id                                                                            4    
uid                                                                                       
019c8ff2-e547-420e-b775-d8089d6b9653                                              house   
07ff7f5a-73ab-4741-b5c3-127c247b79b7  3 connected cubes in a pyramid with a cylinder...   
08d92e08-2e2d-4560-84bf-0b2c352fce93  This is a podium for athletes for standing on ...   
0c6be3bd-e5d0-4deb-90c9-93d53b32c3a7  3 blocks stacked on top of eachother with a cy...   
0d51a401-ef06-493f-bccd-0d0e91a53c6d  It is three squares stuck together with two on...   

                                                                                         \
object_id                                                                            5    
uid                                                                                       
019c8ff2-e547-420e-b775-d8089d6b9653                            Lighthouse on an island   
07ff7f5a-73ab-4741-b5c3-127c247b79b7  This is a smooth mound with a a stack of 3 sha...   
08d92e08-2e2d-4560-84bf-0b2c352fce93  This looks like a mushroom-like object placed ...   
0c6be3bd-e5d0-4deb-90c9-93d53b32c3a7        circular base with triangular levels on top   
0d51a401-ef06-493f-bccd-0d0e91a53c6d               It kind of reminds me of a mushroom.   

                                                                                         \
object_id                                                                            6    
uid                                                                                       
019c8ff2-e547-420e-b775-d8089d6b9653  There is an almost complete circle with a roun...   
07ff7f5a-73ab-4741-b5c3-127c247b79b7  Donut shape with a circular p

In [9]:
# Flatten
result.columns = [
    (
        "description_object_" + str(col[1])
        if col[0] == "description"
        else "seen_object_" + str(col[1])
    )
    for col in result.columns
]
result.reset_index(inplace=True)
result.head()

,uid,description_object_1,description_object_2,description_object_3,description_object_4,description_object_5,description_object_6,description_object_7,description_object_8,description_object_9,...,seen_object_21,seen_object_22,seen_object_23,seen_object_24,seen_object_25,seen_object_26,seen_object_27,seen_object_28,seen_object_29,seen_object_30
0,019c8ff2-e547-420e-b775-d8089d6b9653,Mushroom,bus,dradel,house,Lighthouse on an island,There is an almost complete circle with a roun...,Scarecrow,This looks like a donut on a stick,House,...,yes,yes,no,yes,yes,yes,yes,no,yes,yes
1,07ff7f5a-73ab-4741-b5c3-127c247b79b7,There is an arch on the top of a half of a rec...,sphere with a disk with a cylinder with anothe...,hexagonal prism with 3 triangular prisms on on...,3 connected cubes in a pyramid with a cylinder...,This is a smooth mound with a a stack of 3 sha...,Donut shape with a circular prism in the corne...,Pear shape with a small cylinder base. another...,cylinder with a pentagon cut out of the middle...,"This is a rectangular prism, with a hill shape...",...,no,no,no,no,no,no,no,no,no,no
2,08d92e08-2e2d-4560-84bf-0b2c352fce93,This is a bridge with an alleyway under it.,While it is not exactly similar to the last de...,This object consists of two different sized he...,This is a podium for athletes for standing on ...,This looks like a mushroom-like object placed ...,The object has three components. A 3D triangle...,"This object has three pieces, which consist of...",This looks like a hoop through which other obj...,This is a house.,...,no,yes,yes,yes,yes,no,yes,yes,no,yes
3,0c6be3bd-e5d0-4deb-90c9-93d53b32c3a7,rainbow shape on top of rounded trapezoid base,circular prisms connected by cylinder with rou...,hexagonal shape with smaller hexagon sticking ...,3 blocks stacked on top of eachother with a cy...,circular base with triangular levels on top,round wheel with triangle and blocks,cartoon tree,circle with pentagon cut out on top of black h...,rectangular base with triangular caps,...,yes,no,no,no,no,no,no,no,no,no
4,0d51a401-ef06-493f-bccd-0d0e91a53c6d,It kind of looks like a rainbow.,It looks a dumbell.,It is a hexagon with another smaller hexagon o...,It is three squares stuck together with two on...,It kind of reminds me of a mushroom.,It is a circle with a hole inside of the circl...,It looks like a kind of bird.,It is a circle with a pentagon hole and is on ...,It looks like a house with a roof.,...,yes,yes,no,no,no,no,yes,no,yes,yes


In [10]:
result.to_csv("../data/pivot.csv", index=False)